In [1]:
import phrase_entity_extraction as ph

In [2]:
ph.seq.look_ahead([['ADR_HEAD']])

uuid: input
n_init: <node: (*),(*)>:
active values: []
next values: []

In [3]:
ph.return_max_address(ph.seq, "123 main st")

'123 MAIN ST'

In [24]:
ph.get_markers(ph.seq, "10000 COORS BYPASS NW", ['ADDRESS', 'POBOX', 'SUITE'])

[[0, 1, 1, ['SUITE'], '10000'],
 [0, 2, 2, ['ADDRESS'], '10000 coors'],
 [0, 3, 3, ['ADDRESS'], '10000 coors bypass']]

In [25]:
def get_interpretation(seq, arr_words, arr_search_keys):
    """INPUT: ['123', 'main']"""
    t1_encoded = ph.encode_from_word_list(arr_words)
    #print("encoded: " ,t1_encoded)
    seq.look_ahead(t1_encoded)
    next_values = seq.get_next_values()
    #print("returning ", next_values)
    return [search_key for search_key in arr_search_keys if search_key in next_values]

#get_interpretation(seq, w("123 martin luther king jr"))
get_interpretation(ph.seq, ph.w("suite 100"), ['ADDRESS', 'POBOX', 'SUITE','DIR'])

['SUITE', 'DIR']

In [28]:
def decompose_into_dictionary_words(domain, _seq, types):
    last_length = [-1] * len(domain)
    last_interp = ['']*len(domain)
    #print("Length Domain: ", len(domain))
    for i in range(len(domain)):
        #print("INPUT TO GET_INTERPRETATION: ", domain[:i + 1])
        inter1 = get_interpretation(_seq, domain[:i + 1], types)
        if any([ret_type in types for ret_type in inter1]):
            #print("HIT A")
            last_interp[i] = [hit for hit in inter1 if hit in types]
            last_length[i] = i + 1
            
        if last_length[i] == -1:
            for j in range(i):
                inter2 = get_interpretation(_seq, domain[j + 1:i + 1], types)
                #print("SUB INPUT for INTERPRET: ", domain[j+1:i+1], inter2, last_length[j])
                #if last_length[j] != -1 and any([ret_type in types for ret_type in inter2]):
                if any([ret_type in types for ret_type in inter2]):
                    #print('HIT B')
                    last_interp[i] = [hit for hit in inter2 if hit in types]
                    last_length[i] = i - j
                    break
        #print("last_length: ", last_length)
        #print("last_interp: ", last_interp)
    #return last_length, last_interp
    #print("last_length: ", last_length)
    #print("last_interp: ", last_interp)
    #print("BEGIN DECOMPOSITON PHASE")
    decompositions = []
    components = []
    #if last_length[-1] != -1:
    idx = len(domain) - 1
    while idx >= 0:
        #print("idx: ", idx)
        decompositions.append(last_interp[idx]) #domain[idx + 1 - last_length[idx]:idx + 1])
        components.append((last_interp[idx], " ".join(domain[idx + 1 - last_length[idx]:idx + 1])))
        if last_length[idx] == -1:
            idx -= 1
            continue
        idx -= last_length[idx]
    decompositions = decompositions[::-1]
    components = components[::-1]
    print(decompositions)
    return last_length, last_interp, decompositions, components

decompose_into_dictionary_words(ph.w("1209 HWY 144 N".lower()), ph.seq, ['ADDRESS', 'POBOX', 'SUITE', 'ATTN'])

[['SUITE'], '', ['SUITE']]


([1, -1, 3, 2],
 [['SUITE'], '', ['ADDRESS'], ['SUITE']],
 [['SUITE'], '', ['SUITE']],
 [(['SUITE'], '1209'), ('', ''), (['SUITE'], '144 n')])

In [27]:
def return_max_address2(seq, sent):
    kinds = ['ADDRESS', 'POBOX', 'SUITE']
    decomposition = decompose_into_dictionary_words(ph.w(sent.lower()), seq, kinds)
    found_tuples = decomposition[3]
    max_address = []
    for kindof, value in found_tuples:
        if kindof and kindof[0] in kinds:
            max_address.append(value)
    return " ".join(max_address).upper()
            
return_max_address2(ph.seq, "1209 HWY 144 N")

[['SUITE'], '', ['SUITE']]


'1209 144 N'

In [20]:
ph.get_markers(ph.seq, "10000 COORS BYPASS NW", ['ADDRESS', 'ATTN'])

[[0, 2, 2, ['ADDRESS'], '10000 coors'],
 [0, 3, 3, ['ADDRESS'], '10000 coors bypass']]